In [7]:
import torch
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import collections

In [ ]:
class LeNet(torch.nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        ## 构建子模块
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16*5*5, 120)       ## 全连接层
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, classes)

    def forward(self, x):
        ## 前向传播，拼接子模块
        out = torch.nn.functional.relu(self.conv1(x))
        out = torch.nn.functional.max_pool2d(out, 2)
        out = torch.nn.functional.relu(self.conv2(out))
        out = torch.nn.functional.max_pool2d(out, 2)       ## strides=2，滑动步长
        out = out.view(out.size(0), -1)
        out = torch.nn.functional.relu(self.fc1(out))
        out = torch.nn.functional.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [4]:
class LeNetSequential(torch.nn.Module):
    def __init__(self, classes):
        super(LeNetSequential, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 6, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(6, 16, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(16*5*5, 120),
            torch.nn.ReLU(),
            torch.nn.Linear(120, 84),
            torch.nn.ReLU(),
            torch.nn.Linear(84, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [8]:
class LeNetSequentialOrderDict(torch.nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()

        self.features = torch.nn.Sequential(collections.OrderedDict({
            'conv1': torch.nn.Conv2d(3, 6, 5),
            'relu1': torch.nn.ReLU(inplace=True),
            'pool1': torch.nn.MaxPool2d(kernel_size=2, stride=2),

            'conv2': torch.nn.Conv2d(6, 16, 5),
            'relu2': torch.nn.ReLU(inplace=True),
            'pool2': torch.nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = torch.nn.Sequential(collections.OrderedDict({
            'fc1': torch.nn.Linear(16*5*5, 120),
            'relu3': torch.nn.ReLU(),

            'fc2': torch.nn.Linear(120, 84),
            'relu4': torch.nn.ReLU(inplace=True),

            'fc3': torch.nn.Linear(84, classes),
        }))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [10]:
net = LeNetSequential(classes=2)
#net = LeNetSequentialOrderDict(classes=2)

fake_img = torch.randn((4, 3, 32, 32), dtype=torch.float32)

output = net(fake_img)

print(net)
print(output)

LeNetSequential(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=2, bias=True)
  )
)
tensor([[-0.0431,  0.0305],
        [-0.0468,  0.0428],
        [-0.0629,  0.0434],
        [-0.0531,  0.0364]], grad_fn=<AddmmBackward0>)


In [2]:
class ModuleDict(torch.nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        self.choices = torch.nn.ModuleDict({
            'conv': torch.nn.Conv2d(10, 10, 3),
            'pool': torch.nn.MaxPool2d(3)
        })

        self.activations = torch.nn.ModuleDict({
            'relu': torch.nn.ReLU(),
            'prelu': torch.nn.PReLU()
        })

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x

In [3]:
net = ModuleDict()

fake_img = torch.randn((4, 10, 32, 32))

output = net(fake_img, 'conv', 'relu')

print(output)

tensor([[[[0.0000e+00, 2.9902e-01, 2.6293e-01,  ..., 4.3816e-02,
           0.0000e+00, 0.0000e+00],
          [2.6439e-01, 6.2814e-01, 3.2653e-01,  ..., 0.0000e+00,
           7.8652e-01, 0.0000e+00],
          [2.1924e-01, 0.0000e+00, 4.4759e-01,  ..., 0.0000e+00,
           1.4451e-01, 4.4017e-01],
          ...,
          [5.6913e-01, 0.0000e+00, 6.3030e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [8.8690e-02, 1.4442e-01, 2.3617e-01,  ..., 1.3222e-01,
           0.0000e+00, 4.9368e-01],
          [0.0000e+00, 0.0000e+00, 1.6238e-01,  ..., 6.2266e-01,
           0.0000e+00, 1.7609e-01]],

         [[0.0000e+00, 2.4455e-01, 4.6430e-01,  ..., 1.5563e-01,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 4.9345e-04, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 3.7252e-01],
          [4.4713e-01, 0.0000e+00, 6.9325e-01,  ..., 1.0382e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 6.2994e-01,  ..., 0.0000

In [11]:
class AlexNet(torch.nn.Module):
    
    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [12]:
import torchvision
alexnet = torchvision.models.AlexNet()

D:\Anaconda3\envs\pytorch\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\Anaconda3\envs\pytorch\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
